

# Exploring dataset with chemiscope

The :py:func:`chemiscope.explore` function provides a streamlined way to visualize
datasets as the low dimensional maps. This approach provides a quick and interactive
overview of dataset composition and structure without the need to manually implement and
configure the representation processes. This is particularly useful when the specific
choice of hyperparameters does not significantly impact the resulting low-dimensionality
map.

By passing a list of [ase.Atoms](https://ase-lib.org/ase/ase/atoms.html) objects
(or similar structures from other libraries) to :py:func:`chemiscope.explore`, it is
possible to generate a chemiscope widget, providing an immediate and intuitive
visualization of the dataset.

By default, the method uses the [PETMADFeaturizer](https://arxiv.org/abs/2506.19674),
which computes representations as the [PET-MAD](https://arxiv.org/abs/2503.14118)
features and maps them a low-dimensional MAD dataset latent space.

For more advanced use cases, :py:func:`chemiscope.explore` allows to provide a custom
function for representation and dimensionality reduction.

To use this function, some additional dependencies are required. You can install them
with the following command:

.. code:: bash

    pip install chemiscope[explore]

In this example, we will explore basic and advanced use cases, from simple dataset
visualization to custom featurization.

First, let's import the necessary packages that will be used throughout the examples.


In [ ]:
import ase.io

import chemiscope

## Basic example

This example shows the basic usage of :py:func:`chemiscope.explore`. First, load a
dataset of structures as [ase.Atoms](https://ase-lib.org/ase/ase/atoms.html)
objects. Here, we use the samples from the [MC3D dataset](https://doi.org/10.24435/materialscloud:rw-t0):



In [ ]:
structures = ase.io.read("data/explore_mc3d.xyz", ":")

Next, pass the structures to :py:func:`chemiscope.explore` to generate an interactive
Chemiscope. In this basic case, we provide the featurizer version to be used:



In [ ]:
chemiscope.explore(structures, featurizer="pet-mad-1.0")

We can also save the visualization to send it to collaborators or reopen
separately with :py:func:`chemiscope.show_input`:



In [ ]:
chemiscope.explore(
    structures,
    featurizer="pet-mad-1.0",
    write_input="mc3d.chemiscope.json.gz",
)

Besides this, it is possible to specify atom-centered environments and properties.
Environments can be manually defined as a list of tuples in the format
``(structure_index, atom_index, cutoff)`` or extracted automatically using
:py:func:`chemiscope.all_atomic_environments`. We can also configure visualisation
settings, such as axis and color properties.



In [ ]:
properties = chemiscope.extract_properties(structures, only=["energy"])
environments = [(0, 0, 3.5), (1, 0, 3.5), (2, 1, 3.5)]
settings = chemiscope.quick_settings(
    x="features[1]", y="features[2]", map_color="energy"
)
chemiscope.explore(
    structures,
    featurizer="pet-mad-1.0",
    environments=environments,
    properties=properties,
    settings=settings,
)

## Example with custom featurizer

For advanced use cases, you can define a custom featurization function. For example,
we can describe structures based on their chemical compositions. The function must
take two arguments: ``structures`` and ``environments`` (optional argument for the
atom-centered environments). Below, we create a function to calculate fractional
composition vectors and apply PCA for dimensionality reduction:



In [ ]:
import numpy as np  # noqa
from sklearn.decomposition import PCA  # noqa


def fractional_composition_featurize(structures, environments):
    if environments is not None:
        raise ValueError("'environments' are not supported by this featurizer")

    dimentionality = 100

    features = []

    for structure in structures:
        unique, counts = np.unique(structure.numbers, return_counts=True)
        fractions = counts / len(structure.numbers)

        feature_vector = np.zeros(dimentionality)
        for element_number, franction in zip(unique, fractions, strict=True):
            feature_vector[element_number - 1] = franction

        features.append(feature_vector)

    pca = PCA(n_components=3)
    return pca.fit_transform(features)

Pass the custom featurizer to :py:func:`chemiscope.explore`:



In [ ]:
settings = chemiscope.quick_settings(x="features[1]", y="features[2]")
chemiscope.explore(
    structures,
    featurizer=fractional_composition_featurize,
    settings=settings,
)

For more advanced examples, see the `next tutorial <explore-advanced-example>`.

